In [ ]:
import numpy as np
import os
from tqdm import tqdm
import dataset_utils
import logging
import torch
import torch.optim as optim
from torch.autograd import Variable
from transformers import get_cosine_schedule_with_warmup
import utils
import model.net as net
import model.data_loader as data_loader
from train import train_and_evaluate

In [ ]:
def build_datasets(data_dir, output_dir, train_prop=0.9, config=None):

    assert os.path.isdir(data_dir), "Couldn't find the dataset at {}".format(data_dir)

    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    else:
        raise RuntimeError('{} already exists!'.format(output_dir))
            
    #Intialise a random number generator for reproducible experiments
    rng = np.random.default_rng(config['seed'])

    # Get the filenames in directory
    filenames = os.listdir(data_dir)
    filenames = [os.path.join(data_dir, f) for f in filenames if f.endswith('.txt')]

    # Split the recordings into train and val
    # Make sure to always shuffle with a fixed seed so that the split is reproducible
    filenames.sort()
    rng.shuffle(filenames)

    split = int(train_prop * len(filenames))

    train_filenames = filenames[:split]
    val_filenames = filenames[split:]

    filenames_dict = {'train': train_filenames,
                      'val': val_filenames}

    for split in ['train', 'val']:
        print('{}: {} recordings'.format(split, len(filenames_dict[split])))

    # Preprocess train, val
    for split in ['train', 'val']:
        output_dir_split = os.path.join(output_dir, '{}_npz'.format(split))
        if not os.path.exists(output_dir_split):
            os.makedirs(output_dir_split, exist_ok=True)
        else:
            print("Warning: dir {} already exists".format(output_dir_split))

        print("Processing {} data, saving preprocessed data to {}".format(split, output_dir_split))
        for filename in tqdm(filenames_dict[split]):
            dataset_utils.save_presence(filename, data_dir, output_dir_split, config)
            dataset_utils.save_absence(filename, data_dir, output_dir_split, rng, config)

    print("Done building datasets")


data_dir = 'data/retraining' # recordings with txt labels
output_dir = 'data/retraining_clips' # output directory
config = {'sr': 24000, 'duration': 4.0, 'num_absence': 50, 'pad': 1, 'seed': 12345}
build_datasets(data_dir, output_dir, train_prop=0.9, config=config)

In [ ]:
def batch_to_device(batch, device):
    batch_dict = {key: batch[key].to(device) for key in batch}
    return batch_dict

def get_scheduler(optimizer, total_steps, params):
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=params.warmup * (total_steps // params.batch_size),
        num_training_steps=params.num_epochs * (total_steps // params.batch_size),
    )
    return scheduler

def model_trainer(model_dir, data_dir, restore_file=None):

    # Load the parameters from json file
    json_path = os.path.join(model_dir, 'params.json')
    assert os.path.isfile(
        json_path), "No json configuration file found at {}".format(json_path)
    params = utils.Params(json_path)

    # use GPU if available
    params.cuda = torch.cuda.is_available()

    # Set the random seed for reproducible experiments
    torch.manual_seed(params.seed)
    if params.cuda:
        torch.cuda.manual_seed(params.seed)

    # Set the logger
    utils.set_logger(os.path.join(model_dir, 'train.log'))

    # Create the input data pipeline
    logging.info("Loading the datasets...")

    logging.info("GPU: {}".format(params.cuda))

    # fetch dataloaders
    dataloaders = data_loader.fetch_dataloader(
        ['train', 'val'], data_dir, params)
    train_dl = dataloaders['train']
    val_dl = dataloaders['val']
    logging.info("num_train: {}, num_val: {}".format(len(train_dl.dataset), len(val_dl.dataset)))

    logging.info("- done.")

    # Define the model and optimizer
    model = net.Net(params).cuda() if params.cuda else net.Net(params)
    
    # using Adam defaults from: https://docs.fast.ai/optimizer.html#Adam
    optimizer = optim.AdamW(model.parameters(), lr=params.learning_rate, betas=(0.9, 0.99), eps=1e-05, weight_decay=params.weight_decay)

    total_steps = len(train_dl.dataset)
    scheduler = get_scheduler(optimizer, total_steps, params)

    # fetch loss function and metrics
    loss_fn = net.loss_fn
    metrics = net.metrics

    # Train the model
    logging.info("Starting training for {} epoch(s)".format(params.num_epochs))
    train_and_evaluate(model, train_dl, val_dl, optimizer, scheduler, loss_fn, metrics, params, model_dir,
                       restore_file)


def train_model(model_dir, data_dir):
    # Load the "reference" parameters from parent_dir json file
    json_path = os.path.join(model_dir, 'params.json')
    assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)
    params = utils.Params(json_path)

    model_trainer(model_dir, data_dir)

model_dir = 'experiments/training_model' # needs params.json file
data_dir = 'data/retraining_clips'
train_model(model_dir, data_dir)
